In [1]:
from glob import glob

from collections import defaultdict
import json
import os

In [2]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai9-aiservices1611641965.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ""

engine = 'gpt-35-turbo'

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
questions = defaultdict(set)
q = []
for f in glob('base-questions-kerajaan-*/*'):
    b = os.path.split(f)[0]
    with open(f) as fopen:
        data = json.load(fopen)
    data = '\n'.join(data)
    data = data.split('\n\n')
    if len(data) == 1:
        data = data[0].split('\n')
    else:
        datas = []
        for d in data:
            splitted = d.split('\n ')
            splitted = [s.strip() for s in splitted]
            datas.extend(splitted)
        data = datas
    data = ['.'.join(d.split('.')[1:]).strip() for d in data]
    data = [d for d in data if len(d) > 3]
        
    data = set(data)
    questions[b] |= data

In [4]:
total_questions = 0
all_questions = []
for k, v in questions.items():
    total_questions += len(v)
    all_questions.extend(v)
    
total_questions

31291

In [5]:
all_questions = sorted(all_questions)

In [6]:
all_questions

['Adakah Agrobank memberi tumpuan khas kepada pembiayaan projek-projek yang berkaitan dengan pembangunan infrastruktur di luar bandar di Malaysia?',
 'Adakah Agrobank mempunyai perkhidmatan pengurusan kewangan peribadi yang sesuai untuk orang dewasa muda di Malaysia?',
 'Adakah Agrobank mempunyai program latihan atau bantuan kewangan untuk membantu generasi muda yang berminat dalam sektor pertanian?',
 'Adakah Agrobank mempunyai program pembiayaan khas untuk membantu petani yang terkesan oleh bencana alam seperti banjir atau kebakaran?',
 'Adakah Agrobank menawarkan insurans pertanian kepada petani di Malaysia?',
 'Adakah Agrobank menawarkan perkhidmatan perbankan dalam talian kepada pelanggan-pelanggan mereka?',
 'Adakah Agrobank menawarkan produk pinjaman perumahan yang sesuai untuk pembeli rumah pertama di Malaysia?',
 'Adakah Agrobank menawarkan produk simpanan yang sesuai untuk membantu persaraan pekerja sektor pertanian?',
 'Adakah Agrobank menawarkan skim pembiayaan khas kepada 

In [7]:
!mkdir answer-question-chatgpt3-gov.my

mkdir: cannot create directory ‘answer-question-chatgpt3-gov.my’: File exists


In [8]:
from tqdm import tqdm

for i in tqdm(range((len(all_questions) // 6) * 0, (len(all_questions) // 6) * 1, 1)):
    filename = f'answer-question-chatgpt3-gov.my/{i}.json'
    if os.path.exists(filename):
        continue
        
    try:
        message_text = [
            {"role":"user","content": f"{all_questions[i]}, jawab dalam bahasa melayu"},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-35-turbo",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        splitted = completion.choices[0]['message']['content'].split('\n')
    except:
        splitted = None
    with open(filename, 'w') as fopen:
        json.dump(splitted, fopen)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 5215/5215 [2:58:31<00:00,  2.05s/it]
